<a href="https://colab.research.google.com/github/wrhuang/colab/blob/main/SSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch.autograd import Variable

# dimension
dx = 1000
dz = 100
dy = 10

# sample number
n1 = 10000
n2 = 10000

# pretext data generator
x1 = Variable(torch.randn(dx, n1))
z = x1[:dz, :] + 0.01*torch.normal(torch.zeros(dz, n1),1)

# downstream data generator
x2 = Variable(torch.randn(dx, n2))
y = x2[:dy, :] + 0.01*torch.normal(torch.zeros(dy, n2),1)

In [16]:
# model parameters
wc = Variable(torch.FloatTensor(dz, dx), requires_grad = True)
lamb = 0.1
torch.nn.init.normal_(wc, mean=0, std=1)

tensor([[-1.6097e-02, -6.5981e-01, -9.8760e-01,  ..., -5.9110e-02,
          1.5216e-01,  1.2504e+00],
        [ 1.1570e+00,  7.3599e-01, -8.4568e-01,  ..., -3.9360e-01,
          8.9573e-01,  9.2777e-01],
        [-5.9397e-01, -5.7418e-01,  1.6253e+00,  ..., -5.9980e-01,
         -1.7669e-02,  3.3198e-01],
        ...,
        [-1.4049e+00,  8.6978e-01,  8.9504e-04,  ...,  9.1918e-02,
          3.0082e-02, -1.0484e-01],
        [-1.1414e+00,  4.2471e-02, -5.3894e-01,  ..., -2.4997e-01,
         -1.6463e+00, -1.7725e+00],
        [-2.3001e-01, -8.7770e-01, -5.6376e-01,  ..., -7.8183e-01,
          7.9416e-01,  1.0006e+00]], requires_grad=True)

In [ ]:
epochs = 20000
lr = 10
last_loss = 999

for i in range(epochs):
  i = i+1
  c1 = torch.matmul(wc, x1)
  c1_T = torch.transpose(c1, 0, 1)
  w1 = torch.chain_matmul(torch.pinverse(torch.matmul(c1, c1_T)), c1, torch.transpose(z,0,1))

  c2 = torch.matmul(wc, x2)
  c2_T = torch.transpose(c2, 0, 1)
  w2 = torch.chain_matmul(torch.pinverse(torch.matmul(c2, c2_T)), c2, torch.transpose(y,0,1))

  loss1 = torch.norm(torch.matmul(torch.transpose(c1,0,1),w1)-torch.transpose(z,0,1), dim=1)
  loss2 = torch.norm(torch.matmul(torch.transpose(c2,0,1),w2)-torch.transpose(y,0,1), dim=1)
  loss = torch.mean(loss2) - torch.mean(loss1)*lamb
  
  if i%100==0:
    print(f'epoch={i}, loss={loss}')
  loss.backward()

  with torch.no_grad():
    wc -= lr * wc.grad
    wc.grad.zero_()
  
  if last_loss < loss:
    break
  last_loss = loss



epoch=100, loss=1.8013184070587158
epoch=200, loss=1.4778900146484375
epoch=300, loss=0.9365529417991638
epoch=400, loss=0.19214481115341187
epoch=500, loss=-0.6354119777679443
epoch=600, loss=-0.8744691014289856
epoch=700, loss=-0.8753929138183594
epoch=800, loss=-0.8762966990470886
epoch=900, loss=-0.8771812319755554
epoch=1000, loss=-0.8780466318130493
epoch=1100, loss=-0.8788933753967285
epoch=1200, loss=-0.8797216415405273
epoch=1300, loss=-0.880531907081604
epoch=1400, loss=-0.8813244700431824
epoch=1500, loss=-0.8820995688438416
epoch=1600, loss=-0.8828575611114502
epoch=1700, loss=-0.883599042892456
epoch=1800, loss=-0.8843239545822144
epoch=1900, loss=-0.8850328326225281
epoch=2000, loss=-0.8857259154319763
epoch=2100, loss=-0.8864034414291382
epoch=2200, loss=-0.887066125869751
epoch=2300, loss=-0.8877137899398804
epoch=2400, loss=-0.8883469104766846
epoch=2500, loss=-0.8889659643173218
epoch=2600, loss=-0.8895710706710815
epoch=2700, loss=-0.8901625275611877
epoch=2800, loss

In [16]:
wc.shape

torch.Size([100, 1000])

In [13]:
torch.norm(wc[:,0:9],dim=0)

tensor([8.5263, 9.0020, 8.7312, 9.0478, 9.2518, 8.9538, 9.6542, 9.3752, 9.6234],
       grad_fn=<CopyBackwards>)

In [14]:
torch.norm(wc[:,10:99],dim=0)

tensor([2.8541, 3.0973, 2.9945, 2.6902, 2.9489, 2.8095, 3.4668, 3.1792, 2.9857,
        2.7453, 2.9109, 2.6606, 3.1700, 2.6737, 2.7873, 3.0646, 2.9446, 3.1335,
        2.8279, 2.8779, 3.1074, 3.0287, 3.3657, 2.9438, 3.0469, 2.7410, 3.4830,
        3.5034, 3.1896, 2.6452, 2.9683, 2.7309, 3.2930, 3.2287, 3.0535, 2.6696,
        3.0652, 2.9075, 3.0472, 3.0646, 2.8729, 3.0404, 3.2504, 3.1973, 3.4336,
        3.1482, 2.9991, 3.3161, 2.7615, 3.0308, 2.7638, 3.3246, 3.2844, 3.0035,
        2.7270, 3.2018, 3.1896, 3.0827, 2.7565, 3.3848, 2.9844, 3.1788, 2.7677,
        3.3893, 3.1123, 3.1816, 2.5607, 2.7820, 2.8995, 2.8415, 2.9883, 2.9450,
        2.7592, 2.6565, 3.3650, 2.8204, 2.7743, 3.0673, 2.8414, 3.2894, 3.3344,
        2.7996, 3.1112, 2.9558, 3.0691, 2.9263, 3.0787, 2.9028, 3.4751],
       grad_fn=<CopyBackwards>)

In [15]:
torch.norm(wc[:,100:999],dim=0)

tensor([10.6303, 10.5668, 10.6948, 10.9328, 10.3416, 10.7403, 10.5056, 11.4694,
        10.4560, 10.9550, 11.5283,  9.6686, 11.0914, 10.3341, 11.3453, 10.8139,
        10.3907, 10.5352, 11.4074,  9.1758, 11.6134, 10.7069, 10.3322,  8.9797,
         9.6399, 10.4763, 10.4705, 10.2820,  9.5484,  9.8416,  9.6571, 11.0201,
        10.6916, 10.3296, 10.9799, 10.0771, 10.9470, 10.0145, 10.5150, 10.5857,
        10.4242, 10.1496, 10.0800,  9.2742,  9.9713, 11.0098, 10.7988, 10.6680,
        10.1006, 10.2039,  9.5419, 10.0611,  9.8971,  8.7866, 10.2681, 10.7606,
        11.6471,  9.6636, 11.0798, 10.4409,  9.7128, 10.1463, 11.1101,  9.3904,
        10.1379, 10.8966, 11.4554, 10.2444, 10.4791, 10.7828, 11.0819,  9.8814,
         9.8619, 10.7827, 10.1247,  9.9842, 10.5417, 11.1089, 10.2675, 10.2127,
         9.9870,  9.1812, 10.3596, 10.6315, 10.0574, 10.4935, 11.8075, 10.5540,
         9.7678, 10.3813,  9.9816, 10.4555, 10.4669, 10.1164, 10.2813,  9.6987,
        10.4999, 10.9769, 11.0314,  9.61